In [ ]:
pip install geopy
pip install geopandas
!pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic


In [3]:
# Read observation data
obs = pd.read_excel(r"D:\sowrabh\projects\Numerical Algorithm Developer_Assignment\Observations_001.xlsx")
obs.head()

,S. no.,Freq,PRI,PW,Time,Lat,Lon,Angle,DF_Q
0,1,1804,2008,12.0,18:16:42,21.0146,25,114.643744,1
1,2,1805,2000,14.0,18:02:29,20.3189,25,79.535191,7
2,3,1813,2001,12.0,18:32:36,21.7208,25,118.702706,3
3,4,1804,2004,11.0,18:29:14,21.6241,25,117.312487,3
4,5,1812,2009,10.0,18:00:25,20.0708,25,92.877932,1


In [4]:
features = obs[['Freq', 'PRI', 'PW']].values
features

array([[1.804e+03, 2.008e+03, 1.200e+01],
       [1.805e+03, 2.000e+03, 1.400e+01],
       [1.813e+03, 2.001e+03, 1.200e+01],
       ...,
       [2.094e+03, 3.333e+03, 2.400e+00],
       [2.092e+03, 1.666e+03, 2.400e+00],
       [2.080e+03, 1.666e+03, 2.400e+00]])

In [223]:
import pandas as pd
import numpy as np

# Load data
df_ground_truth = pd.read_excel("Ground_Truth_001.xlsx")
df_observations = pd.read_excel("Observations_001.xlsx")

# --- Helper: expand ranges/lists ---
def parse_range_or_list(value):
    if pd.isna(value):
        return []
    parts = str(value).split(",")
    results = []
    for part in parts:
        part = part.strip()
        if "-" in part:
            start, end = part.split("-")
            results.extend(np.arange(float(start), float(end) + 1))
        else:
            results.append(float(part))
    return results

# Expand GT params
df_ground_truth["Frequency_vals"] = df_ground_truth["Frequency"].apply(parse_range_or_list)
df_ground_truth["PRI_vals"] = df_ground_truth["PRI"].apply(parse_range_or_list)
df_ground_truth["PW_vals"] = df_ground_truth["PW"].apply(parse_range_or_list)

# --- Classification ---
def classify_observation(freq, pri, pw, ground_truth_df):
    for _, row in ground_truth_df.iterrows():
        if (round(freq) in [round(v) for v in row["Frequency_vals"]] and
            round(pri) in [round(v) for v in row["PRI_vals"]] and
            round(pw) in [round(v) for v in row["PW_vals"]]):
            return row["Sno"]
    return None

df_observations["Predicted_ID"] = df_observations.apply(
    lambda x: classify_observation(x["Freq"], x["PRI"], x["PW"], df_ground_truth),
    axis=1
)

# --- Validation ---
# 1. Count how many observations got assigned
assigned_counts = df_observations["Predicted_ID"].value_counts(dropna=False)

# 2. Summarize observed features per radar group
summary = df_observations.groupby("Predicted_ID")[["Freq", "PRI", "PW"]].agg(["mean", "min", "max", "count"])

print("Assigned counts per radar:\n", assigned_counts)
print("\nSummary of classified observations:\n", summary)

# Save results for review
df_observations.to_excel("Observations_001_Classified.xlsx", index=False)


Assigned counts per radar:
 Predicted_ID
1.0     1000
2.0     1000
3.0     1000
4.0     1000
5.0     1000
6.0     1000
8.0     1000
9.0     1000
10.0    1000
7.0      669
NaN      331
Name: count, dtype: int64

Summary of classified observations:
                      Freq                            PRI                    \
                     mean   min   max count         mean   min   max count   
Predicted_ID                                                                 
1.0           1807.536000  1800  1815  1000  2005.036000  2000  2010  1000   
2.0           1834.292000  1825  1843  1000  2589.467000  2281  3007  1000   
3.0           1841.409000  1827  1856  1000  2753.725000  2084  3444  1000   
4.0           1889.480000  1883  1896  1000  1022.176000   990  1054  1000   
5.0           1923.350000  1912  1935  1000   905.890000   872   946  1000   
6.0           1933.826000  1928  1940  1000  1941.220000  1666  2222  1000   
7.0           1997.195815  1980  2014   669  2101.

In [224]:
import pandas as pd
import numpy as np

# --- Load data ---
df_ground_truth = pd.read_excel("Ground_Truth_001.xlsx")
df_observations = pd.read_excel("Observations_001.xlsx")

# --- Helper: expand ranges/lists ---
def parse_range_or_list(value):
    if pd.isna(value):
        return []
    parts = str(value).split(",")
    results = []
    for part in parts:
        part = part.strip()
        if "-" in part:
            start, end = part.split("-")
            results.extend(np.arange(float(start), float(end) + 1))
        else:
            results.append(float(part))
    return results

df_ground_truth["Frequency_vals"] = df_ground_truth["Frequency"].apply(parse_range_or_list)
df_ground_truth["PRI_vals"] = df_ground_truth["PRI"].apply(parse_range_or_list)
df_ground_truth["PW_vals"] = df_ground_truth["PW"].apply(parse_range_or_list)

# --- Classification function ---
def classify_observation(freq, pri, pw, ground_truth_df):
    for _, row in ground_truth_df.iterrows():
        if (round(freq) in [round(v) for v in row["Frequency_vals"]] and
            round(pri) in [round(v) for v in row["PRI_vals"]] and
            round(pw) in [round(v) for v in row["PW_vals"]]):
            return row["Sno"]
    return None

df_observations["Predicted_ID"] = df_observations.apply(
    lambda x: classify_observation(x["Freq"], x["PRI"], x["PW"], df_ground_truth),
    axis=1
)

# --- Summarize classified groups ---
summary = df_observations.groupby("Predicted_ID")[["Freq", "PRI", "PW"]].agg(["mean", "min", "max", "count"])

# --- Automated validation with accuracy scoring ---
validation_rows = []
for rid, stats in summary.iterrows():
    if pd.isna(rid):
        continue
    gt = df_ground_truth[df_ground_truth["Sno"] == rid]
    if gt.empty:
        continue
    
    # Extract GT ranges
    freq_gt = (min(gt.iloc[0]["Frequency_vals"]), max(gt.iloc[0]["Frequency_vals"]))
    pri_gt  = (min(gt.iloc[0]["PRI_vals"]), max(gt.iloc[0]["PRI_vals"]))
    pw_gt   = (min(gt.iloc[0]["PW_vals"]), max(gt.iloc[0]["PW_vals"]))

    # Extract observed ranges
    freq_obs = (stats[("Freq","min")], stats[("Freq","max")])
    pri_obs  = (stats[("PRI","min")], stats[("PRI","max")])
    pw_obs   = (stats[("PW","min")], stats[("PW","max")])

    # Checks
    freq_flag = "OK" if freq_gt[0] <= freq_obs[0] and freq_gt[1] >= freq_obs[1] else "Mismatch"
    pri_flag  = "OK" if pri_gt[0] <= pri_obs[0] and pri_gt[1] >= pri_obs[1] else "Mismatch"
    pw_flag   = "OK" if pw_gt[0] <= pw_obs[0] and pw_gt[1] >= pw_obs[1] else "Mismatch"

    # Accuracy score
    checks = [freq_flag, pri_flag, pw_flag]
    accuracy = (checks.count("OK") / len(checks)) * 100

    validation_rows.append({
        "Radar_ID": rid,
        "Freq_obs_range": freq_obs,
        "Freq_GT_range": freq_gt,
        "Freq_check": freq_flag,
        "PRI_obs_range": pri_obs,
        "PRI_GT_range": pri_gt,
        "PRI_check": pri_flag,
        "PW_obs_range": pw_obs,
        "PW_GT_range": pw_gt,
        "PW_check": pw_flag,
        "Num_Obs": int(stats[("Freq","count")]),
        "Accuracy_%": round(accuracy, 1)
    })

df_validation = pd.DataFrame(validation_rows)

# --- Output ---
print("Validation Results with Accuracy:")
print(df_validation)

# Save for review
df_validation.to_excel("Radar_Classification_Validation.xlsx", index=False)


Validation Results with Accuracy:
   Radar_ID    Freq_obs_range     Freq_GT_range Freq_check     PRI_obs_range  \
0       1.0  (1800.0, 1815.0)  (1800.0, 1815.0)         OK  (2000.0, 2010.0)   
1       2.0  (1825.0, 1843.0)  (1825.0, 1843.0)         OK  (2281.0, 3007.0)   
2       3.0  (1827.0, 1856.0)  (1827.0, 1856.0)         OK  (2084.0, 3444.0)   
3       4.0  (1883.0, 1896.0)  (1883.0, 1896.0)         OK   (990.0, 1054.0)   
4       5.0  (1912.0, 1935.0)  (1912.0, 1935.0)         OK    (872.0, 946.0)   
5       6.0  (1928.0, 1940.0)  (1928.0, 1940.0)         OK  (1666.0, 2222.0)   
6       7.0  (1980.0, 2014.0)  (1980.0, 2014.0)         OK  (2000.0, 2200.0)   
7       8.0  (2010.0, 2030.0)  (2010.0, 2030.0)         OK  (2099.0, 2099.0)   
8       9.0  (2032.0, 2039.0)  (2032.0, 2039.0)         OK  (1642.0, 2400.0)   
9      10.0  (2072.0, 2100.0)  (2072.0, 2100.0)         OK  (1666.0, 3333.0)   

       PRI_GT_range PRI_check  PW_obs_range   PW_GT_range PW_check  Num_Obs  \
0  (20

In [ ]:
#there were no predictive id assigned to some observations
# This could be due to several reasons: 
# 1. The observations may not have enough information to establish a reliable association with a predictive ID.
# 2. There could be errors or noise in the data that prevent accurate matching.
# 3. The model may not have been trained on similar observations, leading to a lack of generalization.
# 4. The threshold for assigning predictive IDs may be too strict, resulting in unassigned observations.


Radar Location Estimation with Error Distances OBSERVATION 001

TRIANGULATION ALGORITHM

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from math import radians, degrees, sin, cos, atan2

# --- Coordinate conversion helpers ---
def geodetic_to_ecef(lat, lon):
    """Convert geodetic lat/lon (degrees) to ECEF coordinates (meters)."""
    R = 6371000  # Earth radius (m)
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

#Bearing is the angular direction or heading from one point to another, measured clockwise from North (0°), where East is 90°, South is 180°, and West is 270°


def bearing_to_unit_vector(lat, lon, bearing):
    """Convert AoA (bearing from north, degrees) into unit direction vector in ECEF."""
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    
    # Local ENU (East, North, Up) basis vectors https://gssc.esa.int/navipedia/index.php/Transformations_between_ECEF_and_ENU_coordinates
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    
    # Direction in ENU (east-north-up)
    dir_enu = sin(bearing) * e + cos(bearing) * n
    
    # Convert ENU direction to ECEF
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    """Estimate radar location by least squares intersection of AoA lines."""
    A = np.zeros((3, 3))
    b = np.zeros(3)
    
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        
        # Projection matrix
        I = np.eye(3)
        P = I - np.outer(d, d)
        
        A += P
        b += P @ p
    
    x = np.linalg.lstsq(A, b, rcond=None)[0]  # least-squares solution
    
    # Convert back to lat/lon
   
    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    
    return lat, lon

# --- Apply to dataset ---
df_ground_truth = pd.read_excel("Ground_Truth_001.xlsx")
df_observations = pd.read_excel("Observations_001_Classified.xlsx")

if "Predicted_ID" not in df_observations.columns:
    raise ValueError("Run Task 1 classification first to add Predicted_ID!")

estimated_locations = []
for radar_id, group in df_observations.groupby("Predicted_ID"):
    if pd.isna(radar_id) or len(group) < 2:
        continue
    est_lat, est_lon = triangulate_location(group)
    estimated_locations.append({"Radar_ID": radar_id, "Est_Lat": est_lat, "Est_Lon": est_lon})

df_estimated = pd.DataFrame(estimated_locations)

# --- Compute error distances against ground truth ---
errors = []
for _, row in df_estimated.iterrows():
    gt = df_ground_truth[df_ground_truth["Sno"] == row["Radar_ID"]]
    if not gt.empty:
        gt_lat, gt_lon = gt.iloc[0]["Latitude"], gt.iloc[0]["Longitude"]
        est_point, gt_point = (row["Est_Lat"], row["Est_Lon"]), (gt_lat, gt_lon)
        error_km = geodesic(est_point, gt_point).km
        errors.append({
            "Radar_ID": row["Radar_ID"],
            "Est_Lat": row["Est_Lat"], 
            "Est_Lon": row["Est_Lon"],
            "GT_Lat": gt_lat, 
            "GT_Lon": gt_lon,
            "Error_km": round(error_km, 2)
        })

df_errors = pd.DataFrame(errors)
print(df_errors)
df_errors.to_excel("Radar_Location_Errors.xlsx", index=False)

# --- Compare with ground truth ---
errors = []
for _, row in df_estimated.iterrows():
    gt = df_ground_truth[df_ground_truth["Sno"] == row["Radar_ID"]]
    if not gt.empty:
        gt_lat, gt_lon = gt.iloc[0]["Latitude"], gt.iloc[0]["Longitude"]
        est_point = (row["Est_Lat"], row["Est_Lon"])
        gt_point = (gt_lat, gt_lon)
        dist_error = geodesic(est_point, gt_point).km
        errors.append({
            "Radar_ID": row["Radar_ID"],
            "Est_Lat": row["Est_Lat"],
            "Est_Lon": row["Est_Lon"],
            "GT_Lat": gt_lat,
            "GT_Lon": gt_lon,
            "Error_km": round(dist_error, 2)
        })



df_errors = pd.DataFrame(errors)

print("Radar Location Estimation with Error Distances:")




   Radar_ID    Est_Lat    Est_Lon   GT_Lat   GT_Lon  Error_km
0       1.0  20.853506  25.788775  19.8775  27.6946    226.42
1       2.0  21.177564  25.749438  20.4273  27.8330    232.27
2       3.0  22.046039  25.853197  21.5086  28.1758    247.47
3       4.0  24.369841  23.641062  24.4546  22.6310    102.87
4       5.0  25.045230  23.444572  25.0335  21.4268    203.63
5       6.0  25.758395  25.957726  26.1642  26.6978     86.69
6       7.0  26.762006  24.282267  26.1381  23.6046     96.68
7       8.0  27.717571  24.209774  27.2327  23.8347     65.28
8       9.0  28.965532  24.094302  28.3128  22.2464    194.62
9      10.0  29.475407  25.977803  29.4965  26.8206     81.77
Radar Location Estimation with Error Distances:


TRIANGULATION ALGORITHM CHECK

In [ ]:
# 
# x is the least-squares solution to the system of equations defined by the observations. It represents the estimated position of the radar in ECEF coordinates.
#    I = np.eye(3)  P = I - np.outer(d, d) A += P b += P @ p
# I is the 3x3 identity matrix. P is the projection matrix that projects points onto the plane perpendicular to the direction vector d. A accumulates these projection matrices, and b accumulates the projected observation points.
#   np.linalg.lstsq(A, b, rcond=None)[0]
# np.linalg.lstsq(A, b, rcond=None)[0] is the least-squares solution to the linear system Ax = b. It finds the x that minimizes the Euclidean 2-norm ||Ax - b||^2.

In [144]:
triangulate_location(df_observations[df_observations["Predicted_ID"] == 10])

(29.47540672667637, 25.97780291975912)

In [220]:
df_estimated = pd.DataFrame(estimated_locations)
df_estimated

,Radar_ID,Est_Lat,Est_Lon
0,1.0,20.853506,25.788775
1,2.0,21.177564,25.749438
2,3.0,22.046039,25.853197
3,4.0,24.369841,23.641062
4,5.0,25.045230,23.444572
5,6.0,25.758395,25.957726
6,7.0,26.762006,24.282267
7,8.0,27.717571,24.209774
8,9.0,28.965532,24.094302
9,10.0,29.475407,25.977803


In [146]:
radar_id

10.0

-------------------------------

CENTROID METHOD

In [ ]:
# import pandas as pd
# import numpy as np
# from geopy.distance import geodesic
# from math import radians, degrees, sin, cos, atan2

# # --- Load data ---
# df_ground_truth = pd.read_excel("Ground_Truth_001.xlsx")
# df_observations = pd.read_excel("Observations_001_Classified.xlsx")

# # --- Helper: bearing to line intersection (least squares) ---
# def bearing_to_vector(lat, lon, bearing):
#     """
#     Convert (lat, lon, bearing) into a direction vector in Cartesian coordinates.
#     """
#     lat, lon, bearing = map(radians, [lat, lon, bearing])
#     # direction vector components
#     dx = cos(lat) * cos(lon)
#     dy = cos(lat) * sin(lon)
#     dz = sin(lat)
#     # bearing vector (simplified for small areas)
#     bx = -sin(bearing)
#     by = cos(bearing)
#     return np.array([dx+bx, dy+by, dz])  # direction

# def estimate_radar_location(observations):
#     """
#     Estimate radar location from multiple (Lat, Lon, AoA).
#     Uses average intersection approach.
#     """
#     lat_mean = np.mean(observations["Lat"].values)
#     lon_mean = np.mean(observations["Lon"].values)
#     return lat_mean, lon_mean  # fallback simple centroid if limited data

# # --- Group observations by predicted radar ID (from Task 1) ---
# if "Predicted_ID" not in df_observations.columns:
#     raise ValueError("Run Task 1 classification first and add Predicted_ID column!")

# estimated_locations = []
# for radar_id, group in df_observations.groupby("Predicted_ID"):
#     if pd.isna(radar_id):
#         continue
#     if len(group) < 2:
#         continue
#     est_lat, est_lon = estimate_radar_location(group)
#     estimated_locations.append({
#         "Radar_ID": radar_id,
#         "Est_Lat": est_lat,
#         "Est_Lon": est_lon
#     })

# df_estimated = pd.DataFrame(estimated_locations)

# # --- Compare with ground truth ---
# errors = []
# for _, row in df_estimated.iterrows():
#     gt = df_ground_truth[df_ground_truth["Sno"] == row["Radar_ID"]]
#     if not gt.empty:
#         gt_lat, gt_lon = gt.iloc[0]["Latitude"], gt.iloc[0]["Longitude"]
#         est_point = (row["Est_Lat"], row["Est_Lon"])
#         gt_point = (gt_lat, gt_lon)
#         dist_error = geodesic(est_point, gt_point).km
#         errors.append({
#             "Radar_ID": row["Radar_ID"],
#             "Est_Lat": row["Est_Lat"],
#             "Est_Lon": row["Est_Lon"],
#             "GT_Lat": gt_lat,
#             "GT_Lon": gt_lon,
#             "Error_km": round(dist_error, 2)
#         })

# df_errors = pd.DataFrame(errors)

# print("Radar Location Estimation with Error Distances:")
# print(df_errors)

# # Save results
# df_errors.to_excel("Radar_Location_Errors.xlsx", index=False)


Radar Location Estimation with Error Distances:
   Radar_ID    Est_Lat  Est_Lon   GT_Lat   GT_Lon  Error_km
0       1.0  21.021944     25.0  19.8775  27.6946    308.39
1       2.0  21.246498     25.0  20.4273  27.8330    308.49
2       3.0  21.997906     25.0  21.5086  28.1758    332.93
3       4.0  23.518633     25.0  24.4546  22.6310    262.41
4       5.0  24.523387     25.0  25.0335  21.4268    365.74
5       6.0  24.578130     25.0  26.1642  26.6978    245.08
6       7.0  27.006850     25.0  26.1381  23.6046    169.09
7       8.0  27.992588     25.0  27.2327  23.8347    142.55
8       9.0  29.031333     25.0  28.3128  22.2464    280.68
9      10.0  28.989062     25.0  29.4965  26.8206    185.70


REMOVED DF_Q 1 or 2

In [213]:
import pandas as pd
from geopy.distance import geodesic

# --- Load again (with classifications already assigned from Task 1) ---
df_ground_truth = pd.read_excel("Ground_Truth_001.xlsx")
df_observations = pd.read_excel("Observations_001_Classified.xlsx")

# Ensure Predicted_ID exists (from Task 1 classification step)
if "Predicted_ID" not in df_observations.columns:
    raise ValueError("Run Task 1 classification first to add Predicted_ID!")

# --- Triangulation function (from Task 2) ---
import numpy as np
from math import radians, degrees, sin, cos, atan2

def geodetic_to_ecef(lat, lon):
    R = 6371000
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

def bearing_to_unit_vector(lat, lon, bearing):
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    dir_enu = sin(bearing) * e + cos(bearing) * n
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    A = np.zeros((3, 3))
    b = np.zeros(3)
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        P = np.eye(3) - np.outer(d, d)
        A += P
        b += P @ p
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    return lat, lon

# --- Step 1: Run estimation with DF_Q filter ---
df_filtered = df_observations[df_observations["DF_Q"].isin([1, 2])]

filtered_locations = []
for radar_id, group in df_filtered.groupby("Predicted_ID"):
    if pd.isna(radar_id) or len(group) < 2:
        continue
    est_lat, est_lon = triangulate_location(group)
    filtered_locations.append({
        "Radar_ID": radar_id,
        "Est_Lat": est_lat,
        "Est_Lon": est_lon
    })

df_filtered_est = pd.DataFrame(filtered_locations)

# --- Step 2: Compare with ground truth ---
filtered_errors = []
for _, row in df_filtered_est.iterrows():
    gt = df_ground_truth[df_ground_truth["Sno"] == row["Radar_ID"]]
    if not gt.empty:
        gt_lat, gt_lon = gt.iloc[0]["Latitude"], gt.iloc[0]["Longitude"]
        est_point, gt_point = (row["Est_Lat"], row["Est_Lon"]), (gt_lat, gt_lon)
        error_km = geodesic(est_point, gt_point).km
        filtered_errors.append({
            "Radar_ID": row["Radar_ID"],
            "Est_Lat": row["Est_Lat"], "Est_Lon": row["Est_Lon"],
            "GT_Lat": gt_lat, "GT_Lon": gt_lon,
            "Error_km_filtered": round(error_km, 2)
        })

df_filtered_errors = pd.DataFrame(filtered_errors)

# --- Step 3: Merge with previous (unfiltered) errors for comparison ---
df_errors = pd.read_excel("Radar_Location_Errors.xlsx")  # from Task 2 output
df_comparison = df_errors.merge(df_filtered_errors, on="Radar_ID", suffixes=("_orig", "_filtered"))

print("Comparison of Location Errors (before vs. after DF_Q filtering):")
print(df_comparison)

# Save
df_comparison.to_excel("Radar_Location_Error_Comparison.xlsx", index=False)


Comparison of Location Errors (before vs. after DF_Q filtering):
   Radar_ID  Est_Lat_orig  Est_Lon_orig  GT_Lat_orig  GT_Lon_orig  Error_km  \
0         1     21.021944            25      19.8775      27.6946    308.39   
1         2     21.246498            25      20.4273      27.8330    308.49   
2         3     21.997906            25      21.5086      28.1758    332.93   
3         4     23.518633            25      24.4546      22.6310    262.41   
4         5     24.523387            25      25.0335      21.4268    365.74   
5         6     24.578130            25      26.1642      26.6978    245.08   
6         7     27.006850            25      26.1381      23.6046    169.09   
7         8     27.992588            25      27.2327      23.8347    142.55   
8         9     29.031333            25      28.3128      22.2464    280.68   
9        10     28.989062            25      29.4965      26.8206    185.70   

   Est_Lat_filtered  Est_Lon_filtered  GT_Lat_filtered  GT_Lon_fi

Observations_011:

Observations_011: Ground truth
■ Number of radars identified and their parameters.
■ Estimated radar locations (Lat/Lon). 
■ Location estimates after excluding DFQ = 1,2 observations. 

Observations_011: Ground truth_011
■ Number of radars identified and their parameters.
■ Estimated radar locations (Lat/Lon). 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from math import radians, degrees, sin, cos, atan2
from geopy.distance import geodesic

# --- Load observations ---
df_observations = pd.read_excel("Observations_011.xlsx")  # repeat for 012

# --- Step 1: Cluster observations by RF parameters, this will help identify distinct radars ---
features = df_observations[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values

# Normalize before clustering
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(features)

# DBSCAN: groups close RF signatures
db = DBSCAN(eps=0.3, min_samples=20).fit(X)

df_observations["Predicted_ID"] = db.labels_  # -1 = noise

# Remove noise (-1) and reindex clusters starting from 1
valid_clusters = sorted([c for c in df_observations["Predicted_ID"].unique() if c != -1])

cluster_map = {old: new+1 for new, old in enumerate(valid_clusters)}

df_observations["Predicted_ID"] = df_observations["Predicted_ID"].map(cluster_map)

# --- Step 2: Triangulation functions (reuse from Task 2) ---
def geodetic_to_ecef(lat, lon):
    R = 6371000
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

def bearing_to_unit_vector(lat, lon, bearing):
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    dir_enu = sin(bearing) * e + cos(bearing) * n
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    A = np.zeros((3, 3))
    b = np.zeros(3)
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        P = np.eye(3) - np.outer(d, d)
        A += P
        b += P @ p
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    return lat, lon

# --- Step 3: Generate Ground Truth ---
generated_ground_truth = []
for radar_id, group in df_observations.groupby("Predicted_ID"):
    if radar_id == -1 or len(group) < 5:  # ignore noise/small groups
        continue
    est_lat, est_lon = triangulate_location(group)
    generated_ground_truth.append({
        "Sno_011": radar_id,
        "Freq_range": f"{group['Freq(MHz)'].min()}-{group['Freq(MHz)'].max()}",
        "PRI_range": f"{group['PRI(usec)'].min()}-{group['PRI(usec)'].max()}",
        "PW_range": f"{group['PW(usec)'].min()}-{group['PW(usec)'].max()}",
        "Est_Lat": round(est_lat, 5),
        "Est_Lon": round(est_lon, 5),
        "Num_Obs": len(group)
    })

df_generated = pd.DataFrame(generated_ground_truth)
# The generated_ground_truth list is populated with dictionaries, each representing a detected radar with its estimated parameters and location like 001.
# --- Save Deliverable ---
df_generated.to_excel("Ground_Truth_011_Generated.xlsx", index=False)
print("Number of radars detected:", len(df_generated))
print(df_generated)

Number of radars detected: 3
   Sno_011 Freq_range  PRI_range   PW_range  Est_Lat   Est_Lon  Num_Obs
0        1    854-899  2000-2200    1.5-1.5 -4.18435  19.86632      510
1        2  1000-2000     10-100    0.1-2.0 -4.00074  20.06131     1307
2        3    854-899  2000-2200  1.94-1.94 -4.17764  19.80184      490


Observations_011:
 
■ Location estimates after excluding DFQ = low quality observations. 

In [236]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from math import radians, degrees, sin, cos, atan2

# --- Load Observations_011 ---
df_obs = pd.read_excel("Observations_011.xlsx")

# --- Step 1: Filter DF_Q values ---
df_filtered = df_obs[~df_obs["DF_Q"].isin([3, 4])]

# --- Step 2: Cluster by RF parameters ---
X = df_filtered[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values
X_scaled = StandardScaler().fit_transform(X)

# DBSCAN clustering (tune eps/min_samples as needed)
db = DBSCAN(eps=0.3, min_samples=20).fit(X_scaled)
df_filtered.loc[:, "Predicted_ID"] = db.labels_  # -1 = noise
# Remove noise (-1) and reindex clusters starting from 1
valid_clusters = sorted([c for c in df_filtered["Predicted_ID"].unique() if c != -1])

cluster_map = {old: new+1 for new, old in enumerate(valid_clusters)}


# When mapping cluster_map, also use .loc:
df_filtered.loc[:, "Predicted_ID"] = df_filtered["Predicted_ID"].map(cluster_map)

# --- Triangulation helpers ---
def geodetic_to_ecef(lat, lon):
    R = 6371000
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

def bearing_to_unit_vector(lat, lon, bearing):
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    dir_enu = sin(bearing) * e + cos(bearing) * n
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    A = np.zeros((3, 3))
    b = np.zeros(3)
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        P = np.eye(3) - np.outer(d, d)
        A += P
        b += P @ p
    x = np.linalg.lstsq(A, b, rcond=None)[0]

    # Convert ECEF back to geodetic coordinates

    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    return lat, lon

# --- Step 3: Estimate radar locations ---
radar_estimates = []
for rid, group in df_filtered.groupby("Predicted_ID"):
    if rid == -1 or len(group) < 10:  # skip noise or very small clusters
        continue
    est_lat, est_lon = triangulate_location(group)
    radar_estimates.append({
        "Sno_011": rid,
        "Freq_range": f"{group['Freq(MHz)'].min()} - {group['Freq(MHz)'].max()}",
        "PRI_range": f"{group['PRI(usec)'].min()} - {group['PRI(usec)'].max()}",
        "PW_range": f"{group['PW(usec)'].min()} - {group['PW(usec)'].max()}",
        "Est_Lat": round(est_lat, 5),
        "Est_Lon": round(est_lon, 5),
        "Num_Obs": len(group)
    })

df_estimates = pd.DataFrame(radar_estimates)

# --- Output ---

print(df_estimates)

# Save results
df_estimates.to_excel("Location_Estimates_011_Filtered.xlsx", index=False)

   Sno_011   Freq_range    PRI_range     PW_range  Est_Lat   Est_Lon  Num_Obs
0        1  1000 - 2000     10 - 100    0.1 - 2.0 -4.01210  20.06419     1088
1        2    854 - 899  2000 - 2200  1.94 - 1.94 -4.13144  19.75173      309
2        3    854 - 899  2000 - 2200    1.5 - 1.5 -4.12404  19.83237      306


C:\Users\sowra\AppData\Local\Temp\ipykernel_13492\1379532894.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, "Predicted_ID"] = db.labels_  # -1 = noise


Compute error change:

ΔLat = Est_Lat_filtered − Est_Lat_unfiltered

ΔLon = Est_Lon_filtered − Est_Lon_unfiltered

ΔDist = geodesic distance difference (km).

In [237]:

import pandas as pd
from geopy.distance import geodesic

def estimate_locations(df):
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import DBSCAN
    from math import radians, degrees, sin, cos

    def geodetic_to_ecef(lat, lon):
        R = 6371000
        lat, lon = radians(lat), radians(lon)
        x = R * cos(lat) * cos(lon)
        y = R * cos(lat) * sin(lon)
        z = R * sin(lat)
        return [x, y, z]

    def bearing_to_unit_vector(lat, lon, bearing):
        lat, lon, bearing = map(radians, [lat, lon, bearing])
        e = [-sin(lon), cos(lon), 0]
        n = [-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)]
        dir_enu = [sin(bearing)*e[i] + cos(bearing)*n[i] for i in range(3)]
        norm = sum(d**2 for d in dir_enu)**0.5
        return [d/norm for d in dir_enu]

    def triangulate_location(observations):
        import numpy as np
        A = np.zeros((3, 3))
        b = np.zeros(3)
        for _, row in observations.iterrows():
            p = geodetic_to_ecef(row["Lat"], row["Lon"])
            d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
            P = np.eye(3) - np.outer(d, d)
            A += P
            b += P @ p
        x = np.linalg.lstsq(A, b, rcond=None)[0]
        lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
        lon = degrees(np.arctan2(x[1], x[0]))
        return lat, lon

    X = df[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values
    X_scaled = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=0.3, min_samples=20).fit(X_scaled)
    df["Radar_ID"] = db.labels_

    results = []
    for rid, group in df.groupby("Radar_ID"):
        if rid == -1 or len(group) < 10:
            continue
        est_lat, est_lon = triangulate_location(group)
        results.append({
            "Radar_ID": rid,
            "Freq_range": f"{group['Freq(MHz)'].min()} - {group['Freq(MHz)'].max()}",
            "PRI_range": f"{group['PRI(usec)'].min()} - {group['PRI(usec)'].max()}",
            "PW_range": f"{group['PW(usec)'].min()} - {group['PW(usec)'].max()}",
            "Est_Lat": round(est_lat, 5),
            "Est_Lon": round(est_lon, 5),
            "Num_Obs": len(group)
        })
    return pd.DataFrame(results)

# --- Load Observations ---
df_obs = pd.read_excel("Observations_011.xlsx")

# Unfiltered estimates
unfiltered = estimate_locations(df_obs)
unfiltered = unfiltered.rename(columns={"Est_Lat": "Est_Lat_Unfiltered", "Est_Lon": "Est_Lon_Unfiltered"})

# DF_Q filtered estimates
df_filtered = df_obs[~df_obs["DF_Q"].isin([3, 4])]
filtered = estimate_locations(df_filtered)
filtered = filtered.rename(columns={"Est_Lat": "Est_Lat_Filtered", "Est_Lon": "Est_Lon_Filtered"})

# Merge results by Radar_ID
comparison = pd.merge(unfiltered, filtered, on="Radar_ID", how="outer")

# Compute change in distance
comparison["Delta_Lat"] = comparison["Est_Lat_Filtered"] - comparison["Est_Lat_Unfiltered"]
comparison["Delta_Lon"] = comparison["Est_Lon_Filtered"] - comparison["Est_Lon_Unfiltered"]
comparison["Delta_Dist_km"] = comparison.apply(
    lambda row: geodesic(
        (row["Est_Lat_Unfiltered"], row["Est_Lon_Unfiltered"]),
        (row["Est_Lat_Filtered"], row["Est_Lon_Filtered"])
    ).km if pd.notnull(row["Est_Lat_Unfiltered"]) and pd.notnull(row["Est_Lat_Filtered"]) else None,
    axis=1
)

# Output results

comparison.to_excel("Location_Estimates_011_Comparison.xlsx", index=False)
print(comparison)
print("Location Estimates for Observations_011 (excluding DF_Q=3,4):")

   Radar_ID Freq_range_x  PRI_range_x   PW_range_x  Est_Lat_Unfiltered  \
0         0    854 - 899  2000 - 2200    1.5 - 1.5            -4.18435   
1         1  1000 - 2000     10 - 100    0.1 - 2.0            -4.00074   
2         2    854 - 899  2000 - 2200  1.94 - 1.94            -4.17764   

   Est_Lon_Unfiltered  Num_Obs_x Freq_range_y  PRI_range_y   PW_range_y  \
0            19.86632        510  1000 - 2000     10 - 100    0.1 - 2.0   
1            20.06131       1307    854 - 899  2000 - 2200  1.94 - 1.94   
2            19.80184        490    854 - 899  2000 - 2200    1.5 - 1.5   

   Est_Lat_Filtered  Est_Lon_Filtered  Num_Obs_y  Delta_Lat  Delta_Lon  \
0          -4.01210          20.06419       1088    0.17225    0.19787   
1          -4.13144          19.75173        309   -0.13070   -0.30958   
2          -4.12404          19.83237        306    0.05360    0.03053   

   Delta_Dist_km  
0      29.077840  
1      37.290754  
2       6.827935  
Location Estimates for Observ

C:\Users\sowra\AppData\Local\Temp\ipykernel_13492\2495376917.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Radar_ID"] = db.labels_


Observations_012

Observations_012: Ground truth_012
■ Number of radars identified and their parameters.
■ Estimated radar locations (Lat/Lon). 

In [256]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from math import radians, degrees, sin, cos, atan2
from geopy.distance import geodesic

# --- Load observations ---
df_observations = pd.read_excel("Observations_012.xlsx")  # repeat for 012

# --- Step 1: Cluster observations by RF parameters, this will help identify distinct radars ---
features = df_observations[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values

# Normalize before clustering
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(features)

# DBSCAN: groups close RF signatures
db = DBSCAN(eps=0.2, min_samples=20).fit(X)

df_observations["Predicted_ID"] = db.labels_  # -1 = noise

# Remove noise (-1) and reindex clusters starting from 1
valid_clusters = sorted([c for c in df_observations["Predicted_ID"].unique() if c != -1])

cluster_map = {old: new+1 for new, old in enumerate(valid_clusters)}

df_observations["Predicted_ID"] = df_observations["Predicted_ID"].map(cluster_map)

# --- Step 2: Triangulation functions (reuse from Task 2) ---
def geodetic_to_ecef(lat, lon):
    R = 6371000
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

def bearing_to_unit_vector(lat, lon, bearing):
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    dir_enu = sin(bearing) * e + cos(bearing) * n
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    A = np.zeros((3, 3))
    b = np.zeros(3)
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        P = np.eye(3) - np.outer(d, d)
        A += P
        b += P @ p
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    return lat, lon

# --- Step 3: Generate Ground Truth ---
generated_ground_truth = []
for radar_id, group in df_observations.groupby("Predicted_ID"):
    if radar_id == -1 or len(group) < 5:  # ignore noise/small groups
        continue
    est_lat, est_lon = triangulate_location(group)
    generated_ground_truth.append({
        "Sno_011": radar_id,
        "Freq_range": f"{group['Freq(MHz)'].min()}-{group['Freq(MHz)'].max()}",
        "PRI_range": f"{group['PRI(usec)'].min()}-{group['PRI(usec)'].max()}",
        "PW_range": f"{group['PW(usec)'].min()}-{group['PW(usec)'].max()}",
        "Est_Lat": round(est_lat, 5),
        "Est_Lon": round(est_lon, 5),
        "Num_Obs": len(group)
    })

df_generated = pd.DataFrame(generated_ground_truth)
# The generated_ground_truth list is populated with dictionaries, each representing a detected radar with its estimated parameters and location like 001.
# --- Save Deliverable ---
df_generated.to_excel("Ground_Truth_012_Generated.xlsx", index=False)
print("Number of radars detected:", len(df_generated))
print(df_generated)

Number of radars detected: 23
    Sno_011 Freq_range  PRI_range     PW_range  Est_Lat   Est_Lon  Num_Obs
0       1.0  1019-1100  1001-2257    2.0-820.0 -3.71769  19.77672     1571
1       2.0  1019-1096  1004-1391   21.0-198.0 -3.70362  19.98432       57
2       3.0  1019-1098  1809-2304  394.0-663.0 -3.80100  20.01658      102
3       4.0  1022-1098  2684-3047  553.0-712.0 -3.70247  19.98198       46
4       5.0  1020-1100  2918-3557   30.0-196.0 -3.73407  20.02951       92
5       6.0  1023-1100  2862-3273  633.0-800.0 -3.67230  20.03082       50
6       7.0  1021-1099  2260-2881    1.0-224.0 -3.73241  19.99405      110
7       8.0  1019-1100  3359-3997  588.0-906.0 -3.78960  19.96736      142
8       9.0  1019-1100  1861-2734  190.0-387.0 -3.78176  19.99232      138
9      10.0  1019-1100  3094-3965  115.0-432.0 -3.82862  19.96455      177
10     11.0  1019-1100  1545-2300  667.0-964.0 -3.74989  20.00573      153
11     12.0  1019-1100  2868-3746  853.0-998.0 -3.80604  19.97226     

Observations_012:
 
■ Location estimates after excluding DFQ = low quality observations. 

In [260]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from math import radians, degrees, sin, cos, atan2

# --- Load Observations_011 ---
df_obs = pd.read_excel("Observations_012.xlsx")

# --- Step 1: Filter DF_Q values ---
df_filtered = df_obs[~df_obs["DF_Q"].isin([1,2])]

# --- Step 2: Cluster by RF parameters ---
X = df_filtered[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values
X_scaled = StandardScaler().fit_transform(X)

# DBSCAN clustering (tune eps/min_samples as needed)
db = DBSCAN(eps=0.2, min_samples=20).fit(X_scaled)
df_filtered.loc[:, "Predicted_ID"] = db.labels_  # -1 = noise
# Remove noise (-1) and reindex clusters starting from 1
valid_clusters = sorted([c for c in df_filtered["Predicted_ID"].unique() if c != -1])

cluster_map = {old: new+1 for new, old in enumerate(valid_clusters)}


# When mapping cluster_map, also use .loc:
df_filtered.loc[:, "Predicted_ID"] = df_filtered["Predicted_ID"].map(cluster_map)

# --- Triangulation helpers ---
def geodetic_to_ecef(lat, lon):
    R = 6371000
    lat, lon = radians(lat), radians(lon)
    x = R * cos(lat) * cos(lon)
    y = R * cos(lat) * sin(lon)
    z = R * sin(lat)
    return np.array([x, y, z])

def bearing_to_unit_vector(lat, lon, bearing):
    lat, lon, bearing = map(radians, [lat, lon, bearing])
    e = np.array([-sin(lon), cos(lon), 0])
    n = np.array([-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)])
    dir_enu = sin(bearing) * e + cos(bearing) * n
    return dir_enu / np.linalg.norm(dir_enu)

def triangulate_location(observations):
    A = np.zeros((3, 3))
    b = np.zeros(3)
    for _, row in observations.iterrows():
        p = geodetic_to_ecef(row["Lat"], row["Lon"])
        d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
        P = np.eye(3) - np.outer(d, d)
        A += P
        b += P @ p
    x = np.linalg.lstsq(A, b, rcond=None)[0]

    # Convert ECEF back to geodetic coordinates

    lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
    lon = degrees(np.arctan2(x[1], x[0]))
    return lat, lon

# --- Step 3: Estimate radar locations ---
radar_estimates = []
for rid, group in df_filtered.groupby("Predicted_ID"):
    if rid == -1 or len(group) < 10:  # skip noise or very small clusters
        continue
    est_lat, est_lon = triangulate_location(group)
    radar_estimates.append({
        "Sno_012": rid,
        "Freq_range": f"{group['Freq(MHz)'].min()} - {group['Freq(MHz)'].max()}",
        "PRI_range": f"{group['PRI(usec)'].min()} - {group['PRI(usec)'].max()}",
        "PW_range": f"{group['PW(usec)'].min()} - {group['PW(usec)'].max()}",
        "Est_Lat": round(est_lat, 5),
        "Est_Lon": round(est_lon, 5),
        "Num_Obs": len(group)
    })

df_estimates = pd.DataFrame(radar_estimates)

# --- Output ---


# Save results
df_estimates.to_excel("Location_Estimates_012_Filtered.xlsx", index=False)

df_estimates

C:\Users\sowra\AppData\Local\Temp\ipykernel_13492\2036823.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1. nan  1. ... 22. 22. 22.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filtered.loc[:, "Predicted_ID"] = df_filtered["Predicted_ID"].map(cluster_map)


,Sno_012,Freq_range,PRI_range,PW_range,Est_Lat,Est_Lon,Num_Obs
0,1.0,1019 - 1100,1001 - 2257,2.0 - 820.0,-3.71769,19.77672,1571
1,2.0,1019 - 1096,1004 - 1391,21.0 - 198.0,-3.70362,19.98432,57
2,3.0,1019 - 1098,1809 - 2304,394.0 - 663.0,-3.80100,20.01658,102
3,4.0,1022 - 1098,2684 - 3047,553.0 - 712.0,-3.70247,19.98198,46
4,5.0,1020 - 1100,2918 - 3557,30.0 - 196.0,-3.73407,20.02951,92
5,6.0,1023 - 1100,2862 - 3273,633.0 - 800.0,-3.67230,20.03082,50
6,7.0,1021 - 1099,2260 - 2881,1.0 - 224.0,-3.73241,19.99405,110
7,8.0,1019 - 1100,3359 - 3997,588.0 - 906.0,-3.78960,19.96736,142
8,9.0,1019 - 1100,1861 - 2734,190.0 - 387.0,-3.78176,19.99232,138
9,10.0,1019 - 1100,3094 - 3965,115.0 - 432.0,-3.82862,19.96455,177


In [ ]:

import pandas as pd
from geopy.distance import geodesic

def estimate_locations(df):
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import DBSCAN
    from math import radians, degrees, sin, cos

    def geodetic_to_ecef(lat, lon):
        R = 6371000
        lat, lon = radians(lat), radians(lon)
        x = R * cos(lat) * cos(lon)
        y = R * cos(lat) * sin(lon)
        z = R * sin(lat)
        return [x, y, z]

    def bearing_to_unit_vector(lat, lon, bearing):
        lat, lon, bearing = map(radians, [lat, lon, bearing])
        e = [-sin(lon), cos(lon), 0]
        n = [-sin(lat)*cos(lon), -sin(lat)*sin(lon), cos(lat)]
        dir_enu = [sin(bearing)*e[i] + cos(bearing)*n[i] for i in range(3)]
        norm = sum(d**2 for d in dir_enu)**0.5
        return [d/norm for d in dir_enu]

    def triangulate_location(observations):
        import numpy as np
        A = np.zeros((3, 3))
        b = np.zeros(3)
        for _, row in observations.iterrows():
            p = geodetic_to_ecef(row["Lat"], row["Lon"])
            d = bearing_to_unit_vector(row["Lat"], row["Lon"], row["Angle"])
            P = np.eye(3) - np.outer(d, d)
            A += P
            b += P @ p
        x = np.linalg.lstsq(A, b, rcond=None)[0]
        
        lat = degrees(np.arcsin(x[2] / np.linalg.norm(x)))
        lon = degrees(np.arctan2(x[1], x[0]))
        return lat, lon

    X = df[["Freq(MHz)", "PRI(usec)", "PW(usec)"]].values
    X_scaled = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=0.2, min_samples=20).fit(X_scaled)
    df["Radar_ID"] = db.labels_

    results = []
    for rid, group in df.groupby("Radar_ID"):
        if rid == -1 or len(group) < 10:
            continue
        est_lat, est_lon = triangulate_location(group)
        results.append({
            "Radar_ID": rid,
            "Freq_range": f"{group['Freq(MHz)'].min()} - {group['Freq(MHz)'].max()}",
            "PRI_range": f"{group['PRI(usec)'].min()} - {group['PRI(usec)'].max()}",
            "PW_range": f"{group['PW(usec)'].min()} - {group['PW(usec)'].max()}",
            "Est_Lat": round(est_lat, 5),
            "Est_Lon": round(est_lon, 5),
            "Num_Obs": len(group)
        })
    return pd.DataFrame(results)

# --- Load Observations ---
df_obs = pd.read_excel("Observations_012.xlsx")

# Unfiltered estimates
unfiltered = estimate_locations(df_obs)
unfiltered = unfiltered.rename(columns={"Est_Lat": "Est_Lat_Unfiltered", "Est_Lon": "Est_Lon_Unfiltered"})

# DF_Q filtered estimates
df_filtered = df_obs[~df_obs["DF_Q"].isin([3])]
filtered = estimate_locations(df_filtered)
filtered = filtered.rename(columns={"Est_Lat": "Est_Lat_Filtered", "Est_Lon": "Est_Lon_Filtered"})

# Merge results by Radar_ID
comparison = pd.merge(unfiltered, filtered, on="Radar_ID", how="outer")

# Compute change in distance
comparison["Delta_Lat"] = comparison["Est_Lat_Filtered"] - comparison["Est_Lat_Unfiltered"]
comparison["Delta_Lon"] = comparison["Est_Lon_Filtered"] - comparison["Est_Lon_Unfiltered"]
comparison["Delta_Dist_km"] = comparison.apply(
    lambda row: geodesic(
        (row["Est_Lat_Unfiltered"], row["Est_Lon_Unfiltered"]),
        (row["Est_Lat_Filtered"], row["Est_Lon_Filtered"])
    ).km if pd.notnull(row["Est_Lat_Unfiltered"]) and pd.notnull(row["Est_Lat_Filtered"]) else None,
    axis=1
)

# Output results

comparison.to_excel("Location_Estimates_012_Comparison.xlsx", index=False)

print("Location Estimates for Observations_012 (excluding DF_Q= 3/4):")
comparison

C:\Users\sowra\AppData\Local\Temp\ipykernel_13492\4164097620.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Radar_ID"] = db.labels_


Location Estimates for Observations_012 (excluding DF_Q= 3/4):


,Radar_ID,Freq_range_x,PRI_range_x,PW_range_x,Est_Lat_Unfiltered,Est_Lon_Unfiltered,Num_Obs_x,Freq_range_y,PRI_range_y,PW_range_y,Est_Lat_Filtered,Est_Lon_Filtered,Num_Obs_y,Delta_Lat,Delta_Lon,Delta_Dist_km
0,0,1019 - 1100,1001 - 2257,2.0 - 820.0,-3.71769,19.77672,1571.0,1019 - 1099,1875 - 2239,3.0 - 147.0,-3.65773,19.44246,831,0.05996,-0.33426,37.720414
1,1,1019 - 1096,1004 - 1391,21.0 - 198.0,-3.70362,19.98432,57.0,1020 - 1098,1897 - 2097,394.0 - 527.0,-3.88592,20.00925,32,-0.18230,0.02493,20.347885
2,2,1019 - 1098,1809 - 2304,394.0 - 663.0,-3.80100,20.01658,102.0,1021 - 1095,1045 - 1312,51.0 - 173.0,-3.80751,20.04659,34,-0.00651,0.03001,3.410231
3,3,1022 - 1098,2684 - 3047,553.0 - 712.0,-3.70247,19.98198,46.0,1024 - 1098,2971 - 3214,666.0 - 789.0,-3.69459,19.90750,28,0.00788,-0.07448,8.319680
4,4,1020 - 1100,2918 - 3557,30.0 - 196.0,-3.73407,20.02951,92.0,1020 - 1097,3789 - 3985,610.0 - 753.0,-3.96385,19.92779,33,-0.22978,-0.10172,27.807522
5,5,1023 - 1100,2862 - 3273,633.0 - 800.0,-3.67230,20.03082,50.0,1023 - 1099,2299 - 2613,33.0 - 167.0,-3.67344,20.01009,37,-0.00114,-0.02073,2.306392
6,6,1021 - 1099,2260 - 2881,1.0 - 224.0,-3.73241,19.99405,110.0,1019 - 1098,1947 - 2243,514.0 - 659.0,-3.72918,19.96892,36,0.00323,-0.02513,2.814326
7,7,1019 - 1100,3359 - 3997,588.0 - 906.0,-3.78960,19.96736,142.0,1019 - 1100,3029 - 3582,867.0 - 996.0,-3.77135,20.01675,64,0.01825,0.04939,5.845582
8,8,1019 - 1100,1861 - 2734,190.0 - 387.0,-3.78176,19.99232,138.0,1021 - 1093,3062 - 3418,65.0 - 152.0,-3.89383,20.06563,31,-0.11207,0.07331,14.828342
9,9,1019 - 1100,3094 - 3965,115.0 - 432.0,-3.82862,19.96455,177.0,1019 - 1099,3359 - 3862,671.0 - 866.0,-3.73698,19.97432,66,0.09164,0.00977,10.191415
